## Practice pandas  

In [1]:

import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("tips.csv")
data.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251


In [3]:
data.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person', 'Payer Name', 'CC Number', 'Payment ID'],
      dtype='object')

### 1. What is the average total bill and tip given by male vs. female customers?

In [4]:
data.groupby("sex")[['total_bill','tip']].mean()

,total_bill,tip
sex,,
Female,18.056897,2.833448
Male,20.744076,3.089618


 ### 2. Number of smokers per day

In [5]:
data[data["smoker"] =='Yes'].groupby('day').size()

day
Fri     15
Sat     42
Sun     19
Thur    17
dtype: int64

### 3. Meal time distribution for parties > 3

In [6]:
data[data['size']>3].groupby("time").count()

,total_bill,tip,sex,smoker,day,size,price_per_person,Payer Name,CC Number,Payment ID
time,,,,,,,,,,
Dinner,37,37,37,37,37,37,37,37,37,37
Lunch,9,9,9,9,9,9,9,9,9,9


### 4.  Average price per person by day

In [7]:
data.groupby("day")['price_per_person'].mean()

day
Fri     8.133158
Sat     8.186782
Sun     7.863684
Thur    7.424194
Name: price_per_person, dtype: float64

### 5. Average tip % of total bill by meal time

In [8]:
data['tip_percent'] = (data['tip']/data['total_bill'])*100

In [9]:
data.groupby('time')['tip_percent'].mean()

time
Dinner    15.951779
Lunch     16.412793
Name: tip_percent, dtype: float64

### 6. Top 3 days by total revenue

In [10]:
data['total_revenue'] = data['total_bill'] + data['tip']

data.groupby("day")['total_revenue'].sum().sort_values(ascending= False).head(3)

day
Sat     2038.80
Sun     1874.55
Thur    1268.16
Name: total_revenue, dtype: float64

### 🔎 7. Filter: bill > $50 and tip < 10%

In [11]:
data[(data['total_bill'] > 50) & (data['tip_percent'] < 20)]

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,tip_percent,total_revenue
170,50.81,10.0,Male,Yes,Sat,Dinner,3,16.94,Gregory Clark,5473850968388236,Sat1954,19.681165,60.81


### 🔎 8. Duplicate payer names

In [12]:
data.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person', 'Payer Name', 'CC Number', 'Payment ID',
       'tip_percent', 'total_revenue'],
      dtype='object')

In [13]:
data[data.duplicated(subset='Payer Name', keep=False)].sort_values('Payer Name')


,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,tip_percent,total_revenue


### 🔐 9. Mask all but last 4 digits of CC Number and Payment ID

In [14]:
data['masked_payment_id']= data['Payment ID'].astype(str).apply(lambda x :"*" *(len(x)-4) + x[-4:])
data['masked_CC_number']= data['CC Number'].astype(str).apply(lambda x :"*" *(len(x)-4) + x[-4:])

In [15]:
data.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,tip_percent,total_revenue,masked_payment_id,masked_CC_number
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,5.944673,18.00,***2959,************3410
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,16.054159,12.00,***4608,************9230
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,16.658734,24.51,***4458,************1322
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,13.978041,26.99,***5260,************5994
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,14.680765,28.20,***2251,************7221


### 10. Which party size tips the most on average, and is it consistent by day?

In [16]:
avg_tip_by_size  = data.groupby('size')['tip'].mean().sort_values(ascending= False)
tip_by_size_day = data.groupby(['day', 'size'])['tip'].mean().unstack().fillna('-')

avg_tip_by_size, tip_by_size_day


(size
 6    5.225000
 4    4.135405
 5    4.028000
 3    3.393158
 2    2.582308
 1    1.437500
 Name: tip, dtype: float64,
 size     1         2         3         4         5    6
 day                                                    
 Fri   1.92  2.644375  3.000000  4.730000         -    -
 Sat    1.0  2.517547  3.797778  4.123846       3.0    -
 Sun      -  2.816923  3.120667  4.087778  4.046667  5.0
 Thur  1.83  2.442500  2.692500  4.218000       5.0  5.3)

### 🔁 11. Pivot table of average tip per gender per day



In [17]:
# Create a pivot table with gender as rows, day as columns, and average tip as values
pd.pivot_table(data, index = "sex", values= 'tip', columns=  'day')



day,Fri,Sat,Sun,Thur
sex,,,,
Female,2.781111,2.801786,3.367222,2.575625
Male,2.693000,3.083898,3.220345,2.980333


### 🧵 12. Extract first and last name from the Payer Name column

In [18]:
data['first_name'] = data['Payer Name'].str.split().str[0]
data['last_name'] = data['Payer Name'].str.split().str[-1]

In [19]:
data.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,tip_percent,total_revenue,masked_payment_id,masked_CC_number,first_name,last_name
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,5.944673,18.00,***2959,************3410,Christy,Cunningham
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,16.054159,12.00,***4608,************9230,Douglas,Tucker
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,16.658734,24.51,***4458,************1322,Travis,Walters
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,13.978041,26.99,***5260,************5994,Nathaniel,Harris
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,14.680765,28.20,***2251,************7221,Tonya,Carter


### 📆 13. Add a fake datetime column and calculate tips by weekday


In [20]:
import numpy as np
import datetime
from datetime import timedelta

In [21]:
data['date'] = pd.to_datetime('2025-05-12') + pd.to_timedelta(np.random.randint(0, 100, size=len(data)), unit='D')

data['weekday'] = data['date'].dt.day_name()

In [22]:
data.groupby('weekday')['tip'].mean().sort_values(ascending=False)

weekday
Monday       3.287941
Thursday     3.086571
Saturday     3.064500
Wednesday    2.960270
Tuesday      2.956897
Friday       2.894750
Sunday       2.693448
Name: tip, dtype: float64

### 🔁 15. Apply a custom function: flag if tip < 10% of bill

In [23]:
def flag_tip(row):
    return "LOW TIP" if row['tip_percent'] <10 else "Good tip"

In [24]:
data['Tip Flag'] = data.apply(flag_tip, axis=1)

In [25]:
data['Tip Flag'].value_counts()

Tip Flag
Good tip    217
LOW TIP      27
Name: count, dtype: int64

 ### Who tipped the most per person?

In [28]:
data[data['price_per_person'] == data['price_per_person'].max()]

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,tip_percent,total_revenue,masked_payment_id,masked_CC_number,first_name,last_name,date,weekday,Tip Flag
184,40.55,3.0,Male,Yes,Sun,Dinner,2,20.27,Stephen Cox,3547798222044029,Sun5140,7.398274,43.55,***5140,************4029,Stephen,Cox,2025-07-20,Sunday,LOW TIP


### Q Which day has the most generous average tip per table size?

In [30]:
data.head(2)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,tip_percent,total_revenue,masked_payment_id,masked_CC_number,first_name,last_name,date,weekday,Tip Flag
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,5.944673,18.0,***2959,************3410,Christy,Cunningham,2025-06-11,Wednesday,LOW TIP
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,16.054159,12.0,***4608,************9230,Douglas,Tucker,2025-07-01,Tuesday,Good tip


In [34]:
data['tip_per_person'] = data['tip']/ data['size']
data.groupby('day')['tip_per_person'].mean().sort_values(ascending=False).head(1)

day
Fri    1.329342
Name: tip_per_person, dtype: float64

Q. Payer Names who used the same CC Number multiple times

In [42]:
cc_counts = data['CC Number'].value_counts()

duplicate_cc = cc_counts[cc_counts > 1].index

data[data['CC Number'].isin(duplicate_cc)][['Payer Name', 'CC Number']].drop_duplicates()


,Payer Name,CC Number
